File: ratings.ipynb\
Author: alexkobz\
Date: 07.05.2024\
Task: DSRFU-275

#### Шаг 0. Импорт и получение токена

In [1]:
import asyncio
import requests
import pandas as pd
import numpy as np
import nest_asyncio; nest_asyncio.apply()
from datetime import datetime as dt, date, timedelta
from functions.divide_chunks import divide_chunks
from math import ceil
from functions.get_date import last_day_month, last_day_month_str, last_work_date_month, last_work_date_month_str

# последний день месяца
REPORT_DATE = last_day_month
REPORT_DATE_STR = last_day_month_str
# увеличиваем количество столбцов и строк для просмотра
pd.set_option('display.max_rows', 200, 'display.max_columns', 200)

In [2]:
from rudata.RuDataDF import RuDataDF
from rudata.RuDataRequest import RuDataRequest
RuDataRequest.set_headers()

Getting token started
Getting token finished


#### Шаг 1. Загрузка stage таблиц 

[FintoolReferenceData](https://docs.efir-net.ru/dh2/#/Info/FintoolReferenceData?id=post-fintoolreferencedata) - Получить расширенный справочник по финансовым инструментам.

In [3]:
FintoolReferenceData = RuDataDF("FintoolReferenceData").df
FintoolReferenceData.head()

FintoolReferenceData started
FintoolReferenceData finished. FintoolReferenceData shape (102423, 186)


,facevalue_m,faceftname_m,sumissuevol_m,summarketvol_m,numcoupons_m,nrdcode,regcode_m,regdate_m,regdistdate_m,begdistdate_m,enddistdate_m,endmtydate_m,daysall_m,fullname_nrd,rpregorg_nrd,rpstate_nrd,issuenumber,currentfacevalue_nrd,isqualified_nrd,bondseries,islombardcbr_nrd,numcoupons_nrd,facevalue_nrd,faceftname_nrd,summarketvol_nrd,regcode_nrd,regdate_nrd,regorg_nrd,begdistdate_nrd,daysall_nrd,enddistdate_nrd,endmtydate_nrd,regdistdate_nrd,sumissuevol_nrd,drqty,isincodebase_nrd,nrdcompcode,shqty,summarketvolorig_nrd,issuername_nrd,planmtydate_nrd,firstcoupondate_nrd,depositary_nrd,registrar_nrd,placeprice_nrd,micexlist_nrd,minlot_nrd,formarketbonds_nrd,registraracctype_nrd,registraracctypedate_nrd,facialacc_nrd,acc_open_date_nrd,fraction_nrd,specialized_depo_nrd,cmp_code_nsd_issuer_nrd,cmp_code_nsd_managing_nrd,ratetypenamerus_nrd,ratetypenameeng_nrd,coupontypename_nrd,base_month,base_year,basis_nrd,sectypenamerus_nrd,sectypenameeng_nrd,secformnamerus_nrd,secformnameeng_nrd,sectypenamebr_nrd,secstaterus_nrd,secstateeng_nrd,placementtype_nrd,share_category_name_nrd,share_category_name_en_nrd,dr_category_name_nrd,dr_category_name_en_nrd,company_state_name_nrd,rate_type_id,cp_type_id,coupon_period_base_id,sec_type_id,sec_form_id,sec_state_id,placement_type_id,share_category_id,dr_category_mn,company_state_mn,sector4212u,sna2008,fullname_en_nrd,seniorityname,fintoolid,moex_code,moex_name,fullname,isin144a,isincode,nickname,progfintoolid,regcode,regdate,regorg,amortisedmty,basis,bondstructuralpar,cfi,coupontype,fintooltype,guaranteeamount,guaranteetype,haveoffer,isconvertible,isguaranteed,maturitygroup,securitization,accruedintcalctype,country,couponperyear,faceftname,facevalue,guarantval,havedefault,numcoupons,status,sumissueval,sumissuevol,summarketval,summarketvol,borrowername,borrowerokpo,borrowersector,borroweruid,borrowerinn,borrowercountry,borrowerregionname,issuername,issuerokpo,issuersector,issueruid,issuerinn,issuercountry,issuerregionname,assetssector,numguarantors,begdistdate,begmtydate,daysall,enddistdate,endmtydate,privatedist,regdistdate,securitytype,securitykind,nominaltype,update_time,isregion,nsd_service,raterevisionsperyear,floatratename,issubordinated,note,listing_level,havecovenant,haverepayment,haveindexedfv,numdecimals,couponperiod,ismatched,firstcoupondate,couponrate,iscomplicated,available_for_unqualified,sec_type_br_code,original_country,qualified_test_id,qualified_test_id_tm,base_asset_fintoolid,isqualified,listing_level_spb,have_mm_moex,have_mm_spb,beg_book_date,seniority,tradesite,current_coupontype,profit_type,counter,rn
0,2.0,2.0,NaN,NaN,NaN,None,0.0,0.0,0.0,0.0,0.0,NaN,NaN,None,None,None,None,NaN,NaN,None,NaN,NaN,NaN,None,NaN,None,None,None,None,NaN,None,None,None,NaN,NaN,None,None,NaN,NaN,None,None,None,None,None,NaN,None,NaN,NaN,None,None,None,None,NaN,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,None,None,None,None,None,None,209185,None,None,"Ventoux CCM Acquisition Corp, Ordinary Shares",None,US92280L1017,Ventoux CCM -US92280L1017,NaN,None,None,None,NaN,None,None,None,None,Акция,None,None,NaN,0.0,NaN,None,None,NaN,US,NaN,USD,0.0001,NaN,NaN,NaN,В обращении,NaN,NaN,NaN,NaN,None,None,None,NaN,None,None,None,Ventoux CCM Acquisition,None,None,120453.0,None,US,None,None,NaN,None,None,NaN,None,None,0.0,None,Обыкн-ая,Неопределен,None,2023-10-12T00:25:07,NaN,Нет информации,NaN,None,NaN,None,NaN,None,None,None,NaN,NaN,None,None,NaN,None,False,None,US,1000,1000,NaN,None,NaN,False,False,None,NaN,102.0,None,None,102423,82863
1,2.0,2.0,NaN,NaN,NaN,None,0.0,0.0,0.0,0.0,0.0,NaN,NaN,None,None,None,None,NaN,NaN,None,NaN,NaN,NaN,None,NaN,None,None,None,None,NaN,None,None,None,NaN,NaN,None,None,NaN,NaN,None,None,None,None,None,NaN,None,NaN,NaN,None,None,None,None,NaN,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,None,None,N

[SecurityRatingTable](https://docs.efir-net.ru/dh2/#/Rating/SecurityRatingTable?id=post-securityratingtable) - Получить рейтинги нескольких бумаг и связанных с ними компаний на заданную дату.

In [4]:
%%time
SecurityRatingTable = RuDataDF("SecurityRatingTable").df
SecurityRatingTable.head()

SecurityRatingTable started
SecurityRatingTable finished. SecurityRatingTable shape (261197, 15)
CPU times: user 18.2 s, sys: 2.51 s, total: 20.8 s
Wall time: 25min 26s


,id,hid,code_name,id_value,forecast,change,date_award,role,okpo,fininstid,fintoolid,isin,id_rating,code,shortname_rus
0,0,9,EXP-Note-Nsc,Снят,None,снят,2023-12-14T00:00:00,0,None,6391.0,129143.0,RU000A0JX2X9,241,RU000A0JX2X9,Башнефть-001P-02R-боб
1,0,57,AKRA-Bnd-Nsc,AA+(RU),None,Подтвержден,2024-09-23T00:00:00,0,None,6317.0,100086.0,RU000A0JX5W4,254,RU000A0JX5W4,Альфа-Банк-18-боб
2,0,911,IFX-Bnd-Isc,Снят,None,снят,2022-03-31T00:00:00,0,None,6317.0,100086.0,RU000A0JX5W4,72,RU000A0JX5W4,Альфа-Банк-18-боб
3,0,912,IFX-Bnd-Nsc,Снят,None,снят,2017-02-06T00:00:00,0,None,6317.0,100086.0,RU000A0JX5W4,73,RU000A0JX5W4,Альфа-Банк-18-боб
4,0,913,IFX-Bnd-Isc-max,Снят,None,снят,2022-03-31T00:00:00,0,None,6317.0,100086.0,RU000A0JX5W4,79,RU000A0JX5W4,Альфа-Банк-18-боб


[ExchangeTree](https://docs.efir-net.ru/dh2/#/Info/ExchangeTree?id=post-exchangetree) - Получить иерархию торговых площадок/источников, используемых Интерфакс

In [5]:
ExchangeTree = RuDataDF("ExchangeTree").df
ExchangeTree.head()

ExchangeTree started
ExchangeTree finished. ExchangeTree shape (239, 10)


,id,id_parent,shortname_rus,shortname_eng,fullname_rus,fullname_eng,sort_order,update_date,counter,rn
0,144,63,Нал,Cash,Наличная валюта,Cash,0,2020-12-28T17:03:09,239,1
1,270,63,RUDIP 2,RUDIP 2,RUDIP 2,RUDIP 2,1,2022-10-31T19:41:24,239,2
2,84,83,США,US,Макроиндикаторы США,USA macroindicators,1,2020-12-28T17:03:09,239,3
3,85,83,РФ,RU,Макроиндикаторы Россия,Russia macroindicators,2,2020-12-28T17:03:09,239,4
4,2,1,РТС,RTS,РТС индексы,RTS indexes,2,2020-12-28T17:03:09,239,5


[ListScaleValues](https://docs.efir-net.ru/dh2/#/Rating/ListScaleValues?id=post-listscalevalues) - Список шкал значений рейтингов

In [6]:
ListScaleValues = RuDataDF("ListScaleValues").df
ListScaleValues.head()

ListScaleValues started
ListScaleValues finished. ListScaleValues shape (2018, 8)


,id,scale_id,rating_value,rating_level,fullname_rus,description,aggregation_type,aggregation_level
0,1,1,AAA(rus),1,Наивысший уровень кредитоспособности,Национальные рейтинги категории «ААА» обознача...,None,NaN
1,2,1,AA+(rus),2,None,None,None,NaN
2,3,1,AA(rus),3,Очень высокая кредитоспособность,Национальные рейтинги категории «AA» обозначаю...,None,NaN
3,4,1,AA-(rus),4,None,None,None,NaN
4,5,1,A+(rus),5,None,None,None,NaN


[ListRatings](https://docs.efir-net.ru/dh2/#/Dictionary/ListRatings?id=post-listratings) - Список рейтингов

In [7]:
ListRatings = RuDataDF("ListRatings").df
ListRatings.head()

ListRatings started
ListRatings finished. ListRatings shape (362, 26)


,id,rating_id,code_name,agency,fullname_rus,fullname_eng,for_instrument,for_company,agency_eng,official_name,is_credit,term_type,term_type_name,currency_type,currency_type_name,scale_type,scale_type_name,is_archive,is_accred_br,doc_511p,doc_421p,doc_180i,agency_id,usedInAgg,scale_id,aggregation_type
0,134,104,MDS-B-Cr&D-LT-KZsc,Мудиз,"Рейтинг банковских депозитов, долгосрочный, Ка...","Bank deposit ratings, long-term, kazakhstan scale",0,1,Moody's,LT Bank Deposits (kaz),1,L,Долгосрочный,L,Локальная,N,Национальная,0,1.0,0,0,0,2,False,22,None
1,135,109,MDS-B-Cr&D-LT-UAsc,Мудиз,"Рейтинг банковских депозитов, долгосрочный, Ук...","Bank deposit ratings, long-term, ukrainian scale",0,1,Moody's,LT Bank Deposits (ukr),1,L,Долгосрочный,L,Локальная,N,Национальная,0,1.0,0,0,0,2,False,23,None
2,451,110,RUS-AM-FS&R-Isc,РусРейтинг,"Рейтинг надежности управляющих компаний, между...","Reliability rating of the management company, ...",0,1,RusRating,Рейтинги надежности управляющих компаний,0,None,None,None,None,I,Международная,1,NaN,0,0,0,6,False,35,None
3,452,111,RUS-AM-FS&R-Nsc,РусРейтинг,"Рейтинг надежности управляющих компаний, нацио...","Reliability rating of the management company, ...",0,1,RusRating,Рейтинги надежности управляющих компаний,0,None,None,L,Локальная,N,Национальная,1,NaN,0,0,0,6,False,36,None
4,446,114,EXP-IN-FS&R-Isc,Эксперт РА,"Рейтинги надежности страховых компаний, междун...","Reliability rating of insurance companies, int...",0,1,Expert RA,Рейтинги финансовой надежности страховых компаний,1,None,None,None,None,I,Международная,1,1.0,0,0,0,4,False,47,None


[Emitents](https://docs.efir-net.ru/dh2/#/Info/Emitents?id=post-emitents) - Получить краткий справочник по эмитентам.

In [8]:
Emitents = RuDataDF("Emitents").df
Emitents.head()

Emitents started
Emitents finished. Emitents shape (41577, 91)


,id_emitent,fininstid,shortname_rus,shortname_eng,inn,okpo,ogrn,code,sector,legal_address,phone,www,update_date,fullname_rus_nrd,fullname_eng_nrd,shortname_rus_nrd,shortname_eng_nrd,company_type_short_name,state_reg_num,state_reg_date,state_reg_name,egrul_date,egrul_organ_name,country,lei_code,bik,fax,e_mail,credit_cmp,is_bank_4_non_resident,swift,country_oksm,country_oksm_cbr,country_name_rus,country_name_eng,region_inn,region_soato,region_name,fullname_rus,sparkid,br_fcsm_reg_code,capital,capital_unit,okved,oecd_lvl,oktmo,okato,post_address,okopf,okogu,have_rating,have_risk,is_finorg,is_control_by_finorg,is_gov_or_cb,on_behalf_of_state,is_subjectrf,is_cis_reg,issuer_nrd,tin,kpp,state_reg_number,market_id_nrd,market_name_rus_nrd,market_name_eng_nrd,sp_rx_entity_id,sp_shortname,mds_shortname,mds_org_id,fch_id,fch_shortname,is_monopoly,is_strategic,ifo_list,isregion,sic,sector4212u,sna2008,reg_code,reg_date,reg_org,note,primary_reg_date,inn_fle,other_tin,other_tin_name,okfs_id,okfs_name,is_branch,counter,rn
0,93627,119953,Brazil Minerals,None,None,None,None,XX030692,None,"C/O Incorp Services Inc., 3773 Howard Hughes P...",None,None,2023-06-16T13:15:10.542755,None,None,None,None,Inc US,None,None,None,None,None,USA,5493006FIK0ZLW06RD76,None,None,None,None,None,None,840.0,840.0,США,United States of America,None,None,None,Brazil Minerals Inc,NaN,None,0.0,None,None,0,None,None,None,None,None,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,None,None,None,None,NaN,None,None,None,None,None,None,None,None,0,0,None,NaN,None,None,None,None,None,RA999999,None,None,None,None,None,None,None,False,41577,33095
1,93955,120161,SEB Denmark,None,None,None,None,XX030895,None,Bernstorffsgade 50 1577 Copenhagen V Denmark,None,https://sebgroup.com/about-us/our-business/our...,2022-11-01T12:41:00.457749,None,None,None,None,Branch DK,None,None,None,None,None,DNK,None,None,None,None,None,None,None,208.0,208.0,ДАНИЯ,Denmark,None,None,None,"Skandinaviska Enskilda Banken AB, Denmark Branch",NaN,None,0.0,None,None,0,None,None,None,None,None,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,None,None,None,None,NaN,None,None,None,None,None,None,None,None,0,0,None,NaN,None,None,None,None,None,None,None,None,None,None,None,None,None,False,41577,33278
2,94034,120162,Midclear,None,None,None,None,XX030906,Финанс.сервисы,"PO Box 11-7971, 16th Floor, Aresco Center, San...",None,None,2024-05-24T18:42:04,None,None,None,None,SAL LB,None,None,None,None,None,LBN,549300OMI3WHG0X9PA58,None,None,None,None,None,None,422.0,422.0,ЛИВАН,Lebanon,None,None,None,Midclear S.A.L.,NaN,None,0.0,None,None,7,None,None,None,None,None,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,None,None,None,None,NaN,None,None,None,None,None,None,None,None,0,0,None,NaN,None,None,None,None,None,RA999999,None,None,None,None,None,None,None,False,41577,33279
3,89678,117750,Green Trace,None,40103309320,None,None,XX028690,None,"Biekensalas str. 6, LV-1004, Riga, Latvia",None,None,2022-02-11T15:05:25,None,None,None,None,SIA (ООО) LV,None,None,None,None,None,LVA,5493002KC87ZH6T8VD75,None,None,None,None,None,None,428.0,428.0,ЛАТВИЯ,Latvia,None,None,None,Green Trace SIA,NaN,None,0.0,None,None,0,None,None,None,None,None,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,None,LV40103309320,None,None,NaN,None,None,None,None,None,None,None,None,0,0,None,NaN,None,None,S2,40103309320,2010-07-26T00:00:00,None,None,None,None,None,None,None,None,False,41577,31079
4,90039,117890,Dongwon Systems,None,1238115163,None,None,XX028776,Прочие,"68, Mabang-ro Seocho-gu, Seoul, South Korea",None,https://www.dongwonsystems.co.kr,2024-12-02T16:12:06.792847,None,None,None,None,None,None,None,None,None,None,KOR,None,None,None,None,None,None,None,410.0,410.0,КОРЕЯ Южная,Korea,None,None,None,Dongwon Systems Corporation,NaN,None,0.0,None,None,0,None,None,None,None,None,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,None,123-81-15163,None,None,NaN,None,None,None,None,None,None,None,None,0,0,None,NaN,None,None,None,123-81-15163,None,None,None,None,None,134111-0011712,Corp Num KR,None,None,False,41577,31211


[OfferorsGuarants](https://docs.efir-net.ru/dh2/#/Bond/OfferorsGuarants?id=post-offerorsguarants) - Возвращает список гарантов/оферентов для инструмента

In [9]:
OfferorsGuarants = RuDataDF("OfferorsGuarants").df
OfferorsGuarants.head()

OfferorsGuarants started
OfferorsGuarants finished. OfferorsGuarants shape (5494, 5)


,fintoolId,fininstid_guarant,fininstid_offeror,counter,rn
0,123729,95651,NaN,5494,101
1,123875,95272,NaN,5494,102
2,123926,95133,NaN,5494,103
3,123939,102630,NaN,5494,104
4,123978,102027,NaN,5494,105


[CompanyRatingsTable](https://docs.efir-net.ru/dh2/#/Rating/CompanyRatingsTable?id=post-companyratingstable) - Получить рейтинги нескольких компаний на заданную дату.

In [10]:
CompanyRatingsTable = RuDataDF("CompanyRatingsTable").df
CompanyRatingsTable.head()

CompanyRatingsTable started
CompanyRatingsTable finished. CompanyRatingsTable shape (17668, 13)


,name_ra,dt,last,last_dt,change,forecast,advanced,prev,prev_dt,id_rating,code,code_type,company_name
0,Рейтинг отозван,2017-10-06T00:00:00,Снят,2017-10-06T00:00:00,снят,None,Основной,ruA-,2017-06-07T00:00:00,100,97134,FININSTID,АРТ
1,Рейтинг отозван,2023-11-15T00:00:00,Снят,2023-11-15T00:00:00,снят,None,Основной,bbb-(ru),2022-11-21T00:00:00,200,96904,FININSTID,Техно Лизинг
2,Рейтинг отозван,2013-05-16T00:00:00,Снят,2013-05-16T00:00:00,снят,None,Основной,B3.ru,2013-05-15T00:00:00,133,79248,FININSTID,Юниаструм Банк
3,Рейтинг подтвержден,2023-05-12T00:00:00,BBB,2019-05-21T00:00:00,Подтвержден,Стабильный,Основной,BBB (low),2018-05-24T00:00:00,144,95842,FININSTID,Kinder Morgan
4,Рейтинг отозван,2022-05-13T00:00:00,Снят,2022-05-13T00:00:00,снят,None,Основной,BBB,2021-05-13T00:00:00,148,95842,FININSTID,Kinder Morgan


#### Шаг 2. Предобработка таблиц до экспорта в excel

In [11]:
#добавляем информацию из справочника в нашу таблицу с компаниями
ExtendedSecurityRatingTable = SecurityRatingTable.merge(ListRatings[['rating_id','agency','official_name','term_type_name','scale_type_name','scale_id']], how='left', left_on='id_rating', right_on='rating_id')
ExtendedSecurityRatingTable = ExtendedSecurityRatingTable.merge(ListScaleValues, how='left', left_on=['scale_id', 'id_value'], right_on=['scale_id', 'rating_value'])
ExtendedSecurityRatingTable = ExtendedSecurityRatingTable[[
    'id_x', 'hid', 'code_name', 'id_value', 'forecast', 'change',
    'date_award', 'role', 'okpo', 'fininstid', 'fintoolid', 'isin',
    'id_rating', 'code', 'shortname_rus', 'rating_id', 'agency',
    'official_name', 'term_type_name', 'scale_type_name', 'scale_id',
    'aggregation_level', 'aggregation_type']].rename(
    columns={
    'id_x': 'id', 'aggregation_level': 'level', 'aggregation_type': 'agg_type'
    }
)
agencies = ['АКРА', 'Эксперт РА', 'НКР', 'НРА'
           # ]
            , 'Мудис Интерфакс']
ExtendedSecurityRatingTable = ExtendedSecurityRatingTable[ExtendedSecurityRatingTable['agency'].isin(agencies)]
ExtendedSecurityRatingTable = ExtendedSecurityRatingTable[ExtendedSecurityRatingTable['term_type_name'] != 'Краткосрочный']
ExtendedSecurityRatingTable = ExtendedSecurityRatingTable[~ExtendedSecurityRatingTable['id_value'].isin(['****','Приостановлен','Снят'])]
ExtendedSecurityRatingTable = ExtendedSecurityRatingTable[~ExtendedSecurityRatingTable['isin'].isna()]


In [12]:
akra_list = [
'Structured Finance Instrument Rating',
 'Рейтинг эмиссии, международная шкала',
 'Старший необеспеченный рейтинг эмиссии',
 'Старший обеспеченный рейтинг эмиссии',
 'Субординированный рейтинг эмиссии'
]
expert_list = [
    'Structured finance instrument rating',
    'Кредитный рейтинг долговых инструментов'
]
nkr_list = [
   # 'Кредитный рейтинг долговых обязательств'
]
nra_list = [
 #   'Кредитный рейтинг отдельных выпусков облигаций',
  #  'Рейтинг кредитоспособности выпусков облигаций Индивидуальный'
]
moodys_list = [
    'NSR Senior Unsecured (Domestic)',
 'Senior Unsecured (Domestic); Long-Term Corporate Obligation Rating (Domestic); Structured Finance Long-Term Rating (Domestic)',
 'Senior Unsecured (Foreign); Long-Term Corporate Obligation Rating (Foreign); Structured Finance Long-Term Rating (Foreign)',
 'Subordinate (Domestic)',
 'Subordinate (Foreign)'
]

ExtendedSecurityRatingTable = ExtendedSecurityRatingTable[
((ExtendedSecurityRatingTable['agency']=='АКРА') & (ExtendedSecurityRatingTable['official_name'].isin(akra_list))) |
((ExtendedSecurityRatingTable['agency']=='Эксперт РА') & (ExtendedSecurityRatingTable['official_name'].isin(expert_list))) |
((ExtendedSecurityRatingTable['agency']=='НКР') & (ExtendedSecurityRatingTable['official_name'].isin(nkr_list))) |
((ExtendedSecurityRatingTable['agency']=='НРА') & (ExtendedSecurityRatingTable['official_name'].isin(nra_list))) |
((ExtendedSecurityRatingTable['agency']=='Мудис Интерфакс') & (ExtendedSecurityRatingTable['official_name'].isin(moodys_list)))
 ]

ExtendedSecurityRatingTable = ExtendedSecurityRatingTable\
.dropna(subset=['isin'])\
.drop_duplicates(subset=["isin", "agency"])\
.reset_index(drop=True)

In [13]:
ExtendedSecurityRatingTablePivot = ExtendedSecurityRatingTable.drop_duplicates().pivot(index='isin', columns='agency', values='id_value')
ExtendedSecurityRatingTablePivot.columns = pd.MultiIndex.from_tuples([('Эмиссия', i) for i in ExtendedSecurityRatingTablePivot.columns])

In [ ]:
TransformedFintoolReferenceData = FintoolReferenceData.copy()[['isincode', 'nickname', 'fintooltype', 'country', 'faceftname', 'fintoolid',
                   'issuername', 'issuercountry', 'issuerinn', 'issueruid',
                   'borrowername', 'borrowercountry', 'borrowerinn', 'borroweruid',
                   'isguaranteed', 'guaranteetype', 'guaranteeamount', 'guarantval',
                   ]].drop_duplicates(subset=['isincode'])
TransformedFintoolReferenceData[['fintoolid', 'issueruid', 'borroweruid']] = TransformedFintoolReferenceData[['fintoolid', 'issueruid', 'borroweruid']].fillna(0).astype('int')
TransformedFintoolReferenceData = TransformedFintoolReferenceData.merge(OfferorsGuarants[['fintoolId','fininstid_guarant']], how='left', left_on='fintoolid', right_on='fintoolId')
ids = list(TransformedFintoolReferenceData.loc[~TransformedFintoolReferenceData['fininstid_guarant'].isna(),'fininstid_guarant'].astype('int'))
TransformedEmitents = Emitents[Emitents.fininstid.isin(ids)][['shortname_rus','country','inn','fininstid']]
TransformedEmitents.columns = ['name_guarant','country_guarant','inn_guarant','fininstid']
TransformedFintoolReferenceData = TransformedFintoolReferenceData.merge(TransformedEmitents, how='left', left_on='fininstid_guarant', right_on='fininstid')
TransformedFintoolReferenceData = TransformedFintoolReferenceData.drop(columns=['fintoolId', 'fininstid'])
TransformedFintoolReferenceData = TransformedFintoolReferenceData.set_index('isincode').rename_axis('ISIN')
TransformedFintoolReferenceData = TransformedFintoolReferenceData.rename(
    columns={
        'nickname': ('Эмиссия','Название'),
        'fintooltype': ('Эмиссия','Тип'),
        'country': ('Эмиссия','Страна'), 
        'faceftname': ('Эмиссия','Валюта'), 
        'fintoolid': ('Эмиссия','Id RuData'),
        'issuername': ('Эмитент','Название'), 
        'issuercountry': ('Эмитент','Страна'), 
        'issuerinn': ('Эмитент','ИНН'), 
        'issueruid': ('Эмитент','Id RuData'), 
        'borrowername': ('Заемщик','Название'),
        'borrowercountry': ('Заемщик','Страна'), 
        'borrowerinn': ('Заемщик','ИНН'), 
        'borroweruid': ('Заемщик','Id RuData'), 
        'isguaranteed': ('Гарант','Есть гарантия'),
        'guaranteetype': ('Гарант','Вид гарантии'), 
        'guaranteeamount': ('Гарант','Полнота гарантии'), 
        'guarantval': ('Гарант','Гарантированная сумма'), 
        'fininstid_guarant': ('Гарант','Id RuData'),
        'name_guarant': ('Гарант','Название'), 
        'country_guarant': ('Гарант','Страна'), 
        'inn_guarant': ('Гарант','ИНН')
    }
)
TransformedFintoolReferenceData.columns = pd.MultiIndex.from_tuples(TransformedFintoolReferenceData.columns)

In [ ]:
ExtendedFintoolReferenceData = TransformedFintoolReferenceData.join(ExtendedSecurityRatingTablePivot)
new_cols = ExtendedFintoolReferenceData.columns.reindex(['Эмиссия','Эмитент','Заемщик','Гарант'], level=0)
ExtendedFintoolReferenceData = ExtendedFintoolReferenceData.reindex(columns=new_cols[0])

In [ ]:
ExtendedCompanyRatingsTable = CompanyRatingsTable.merge(ListRatings[['id','agency','official_name','term_type_name','scale_type_name','scale_id']], how='left', left_on='id_rating', right_on='id')
##добавляем уровень рейтинг
ExtendedCompanyRatingsTable = ExtendedCompanyRatingsTable.merge(ListScaleValues, how='left', left_on=['scale_id', 'last'], right_on=['scale_id', 'rating_value'])

In [ ]:
ExtendedCompanyRatingsTable = ExtendedCompanyRatingsTable[['name_ra', 'dt', 'last', 'last_dt', 'change', 'forecast', 'advanced',
           'prev', 'prev_dt', 'id_rating', 'code', 'code_type', 'company_name',
           'id_x', 'agency', 'official_name', 'term_type_name', 'scale_type_name',
           'scale_id', 'aggregation_level', 'aggregation_type']].rename(columns={'id_x': 'id', 'aggregation_level': 'level', 'aggregation_type': 'agg_type'})
ExtendedCompanyRatingsTable['id_rating'] = ExtendedCompanyRatingsTable['id_rating'].fillna(0)

In [ ]:
ExtendedCompanyRatingsTable = ExtendedCompanyRatingsTable[ExtendedCompanyRatingsTable['agency'].isin(['АКРА','Эксперт РА','НКР','НРА','Мудис Интерфакс'])]
ExtendedCompanyRatingsTable = ExtendedCompanyRatingsTable[~ExtendedCompanyRatingsTable['last'].isin(['****','Приостановлен','Снят'])]
ExtendedCompanyRatingsTable = ExtendedCompanyRatingsTable[ExtendedCompanyRatingsTable['term_type_name'] != 'Краткосрочный']

In [ ]:
akra_list=[
 'Рейтинг эмитентов',
 'Long-term credit rating',
 'Long-term local currency credit rating',
 'Long-term foreign currency credit rating',
]
expert_list=[
 'Рейтинг кредитоспособности организаций, осуществляющих микрофинансовую деятельность (Республика Казахстан)',
 'Рейтинг кредитоспособности проектных компаний',
 'Рейтинги кредитоспособности банка Республики Беларусь',
 'Рейтинги кредитоспособности банков',
 'Рейтинги кредитоспособности депозитариев',
 'Рейтинги кредитоспособности лизинговых компаний',
 'Рейтинги кредитоспособности микрофинансовых организаций',
 'Рейтинги кредитоспособности нефинансовых компаний',
 'Рейтинги кредитоспособности нефинансовых компаний Республики Беларусь',
 'Рейтинги кредитоспособности нефинансовых компаний Республики Казахстан',
 'Рейтинги кредитоспособности субъектов федерации и муниципальных образований',
 'Рейтинги кредитоспособности факторинговых компаний',
 'Рейтинги кредитоспособности финансовых компаний',
 'Рейтинги кредитоспособности холдинговых компаний',
 'Рейтинги надежности компаний по страхованию жизни',
 'Рейтинги финансовой надежности негосударственных пенсионных фондов',
 'Рейтинги финансовой надежности страховых компаний'
]
nkr_list=[
 'Кредитный рейтинг НКР по Национальной шкале, Банки',
 'Кредитный рейтинг НКР по Национальной шкале, Нефинансовые компании'
]
nra_list=[
 'Кредитный рейтинг кредитных организаций',
 'Кредитный рейтинг нефинансовых компаний, national scale',
]
moodys_list=[
 'BACKED LT Bank Deposits',
 'BACKED LT Bank Deposits (Domestic)',
 'BACKED LT Bank Deposits (Foreign)',
 'BACKED LT Counterparty Risk Rating',
 'BACKED LT Counterparty Risk Rating (Domestic)',
 'BACKED LT Counterparty Risk Rating (Foreign)',
 'BACKED LT Issuer Rating',
 'BACKED LT Issuer Rating (Domestic)',
 'BACKED LT Issuer Rating (Foreign)',
 'BACKED Senior Unsecured (Domestic)',
 'BACKED Senior Unsecured (Foreign)',
 'Issuer Ratings',
 'LT Bank Deposits',
 'LT Bank Deposits (Domestic)',
 'LT Bank Deposits (Foreign)',
 'LT Bank Deposits (kaz)',
 'LT Bank Deposits (ukr)',
 'LT Corporate Family Ratings',
 'LT Corporate Family Ratings (Domestic)',
 'LT Corporate Family Ratings (Foreign)',
 'LT Corporate Family Ratings (ukr)',
 'LT Counterparty Risk Rating (Domestic)',
 'LT Counterparty Risk Rating (Foreign)',
 'LT Insurance Financial Strength',
 'LT Insurance Financial Strength (Domestic)',
 'LT Insurance Financial Strength (Foreign)',
 'LT Issuer Rating',
 'LT Issuer Rating (Domestic)',
 'LT Issuer Rating (Foreign)',
 'Senior Unsecured',
 'Senior Unsecured (Domestic)',
 'Senior Unsecured (Foreign)',
]

ExtendedCompanyRatingsTable = ExtendedCompanyRatingsTable[
((ExtendedCompanyRatingsTable['agency']=='АКРА') & (ExtendedCompanyRatingsTable['official_name'].isin(akra_list))) |
((ExtendedCompanyRatingsTable['agency']=='Эксперт РА') & (ExtendedCompanyRatingsTable['official_name'].isin(expert_list))) |
((ExtendedCompanyRatingsTable['agency']=='НКР') & (ExtendedCompanyRatingsTable['official_name'].isin(nkr_list))) |
((ExtendedCompanyRatingsTable['agency']=='НРА') & (ExtendedCompanyRatingsTable['official_name'].isin(nra_list))) |
((ExtendedCompanyRatingsTable['agency']=='Мудис Интерфакс') & (ExtendedCompanyRatingsTable['official_name'].isin(moodys_list)))
]

In [ ]:
ExtendedCompanyRatingsTable = ExtendedCompanyRatingsTable.dropna(subset=['code'])
ExtendedCompanyRatingsTable = ExtendedCompanyRatingsTable.reset_index(drop=True)

In [ ]:
for n, g in ExtendedCompanyRatingsTable.groupby(['code', 'agency']):
    if g.shape[0] > 1: 
        try:
            m = g['level'].idxmin(skipna=False)
            if pd.isna(m):
                m = g.index[0]
            to_drop = g.index.drop(m)
            ExtendedCompanyRatingsTable = ExtendedCompanyRatingsTable.drop(index=to_drop)
        except:
            print(g)
            raise
ExtendedCompanyRatingsTable = ExtendedCompanyRatingsTable.dropna(subset=['code'])
ExtendedCompanyRatingsTable = ExtendedCompanyRatingsTable.reset_index(drop=True)
ExtendedCompanyRatingsTable['code'] = ExtendedCompanyRatingsTable['code'].astype(int)

In [ ]:
ExtendedCompanyRatingsPivot = ExtendedCompanyRatingsTable.pivot(index='code', columns='agency', values='last')

In [ ]:
df1 = ExtendedFintoolReferenceData.copy()
df1.columns =df1.columns.to_flat_index()
for k in ['Эмитент','Заемщик','Гарант']:
    df2 = ExtendedCompanyRatingsPivot.copy()
    df2.columns = [(k, i) for i in df2.columns]
    df1 = df1.merge(df2, how='left', left_on=[(k, 'Id RuData')], right_index=True)
df1.columns = pd.MultiIndex.from_tuples(df1.columns)
new_cols = df1.columns.reindex(['Эмиссия', 'Эмитент', 'Заемщик', 'Гарант'], level=0)
df1 = df1.reindex(columns=new_cols[0])
ExtendedFintoolReferenceData = df1

In [ ]:
df = Emitents[Emitents['fininstid'].isin(ExtendedCompanyRatingsPivot.index)]
CompanyRatings = df.merge(ExtendedCompanyRatingsPivot, left_on='fininstid', right_index=True, how='left')

#### Шаг 3. Рейтинги облигаций из фактовой таблицы бумаг FintoolReferenceData. Рейтинги облигаций из фактовой таблицы компаний CompanyRatingsTable

In [ ]:
BondRatings = ExtendedFintoolReferenceData['Эмиссия'].reset_index()
id_vars = BondRatings.columns[:6]
value_vars = BondRatings.columns[6:]
meltBonds = pd.melt(BondRatings, id_vars=id_vars, value_vars=value_vars, var_name='agency', value_name='rate')
meltBonds = meltBonds[~meltBonds.rate.isna()].sort_values('Id RuData')
meltBonds = meltBonds[meltBonds["Тип"] == 'Облигация']

In [ ]:
CompanyRatings = pd.concat([
    CompanyRatings[["id_emitent", "shortname_rus", "inn", "update_date", "country_oksm", "have_rating", "have_risk", "tin", "fininstid", "reg_code", "lei_code"]]
    , CompanyRatings.iloc[:, CompanyRatings.columns.get_loc('rn')+1:]
], axis=1)
id_vars = CompanyRatings.columns[:11]
value_vars = CompanyRatings.columns[11:]
meltCompanies = pd.melt(CompanyRatings, id_vars=id_vars, value_vars=value_vars, var_name='agency', value_name='rate')
meltCompanies = meltCompanies[meltCompanies['rate'].notnull()].sort_values('id_emitent')
meltCompanies['inn'] = np.where(meltCompanies['country_oksm'] == 643, meltCompanies['inn'], meltCompanies['tin'] )
meltCompanies['country_oksm'] = meltCompanies['country_oksm'].astype(np.int32)
meltCompanies['type'] = "ЮЛ"

In [ ]:
meltBonds.rename(columns={
    "Id RuData": "id_emitent",
    "Название": "company_name", 
    "Тип": "rate_type",
    "ISIN": "inn/isin",
    "Страна": "country"
    }, inplace=True)
meltCompanies.rename(columns={
    "id_rudata": "id_emitent",
    "shortname_rus": "company_name", 
    "type": "rate_type",
    "inn": "inn/isin",
    "country_oksm": "country_code"
    }, inplace=True)
meltBonds["fininstid"] = np.nan
meltBonds["reg_code"] = np.nan
meltBonds["lei_code"] = np.nan
meltBonds["report_date"] = pd.to_datetime(REPORT_DATE)
meltCompanies["report_date"] = pd.to_datetime(REPORT_DATE)

Справочник кодов стран ОКСМ

In [ ]:
countries = pd.read_sql(
    """
    SELECT DISTINCT country, alpha2
    FROM countries
    """
    , RuDataDF.engine)
countries["country"] = countries["country"].astype(np.int32)
co = countries.set_index("alpha2").country.to_dict()
meltBonds["country_code"] = meltBonds["country"].map(co)

#### Шаг 4. Получаем результирующий результат по рейтингам облигаций и компаний и экспорт в excel

In [ ]:
meltBonds = meltBonds[["id_emitent", "fininstid", "company_name", "inn/isin", "country_code", "agency", "rate", "report_date", "rate_type", "reg_code", "lei_code"]]
meltCompanies = meltCompanies[["id_emitent", "fininstid", "company_name", "inn/isin", "country_code", "agency", "rate", "report_date", "rate_type", "reg_code", "lei_code"]]
ResultRatings = pd.concat([meltBonds, meltCompanies], axis=0)
with pd.ExcelWriter('data/Output/ratings.xlsx', datetime_format='dd.mm.yyyy') as writer:
    ResultRatings.to_excel(writer, index=False)

In [ ]:
from functions.send_email import main
main("ratings.xlsx")